In [1]:
import csv
import time
import datetime
import pandas as pd 
from pandas.io.json import json_normalize
import json  
import threading
from datetime import datetime
import numpy as np
import glob

In [12]:
df = pd.read_json('./merge_new.json')
len(df)

17458

In [13]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
#df.head(5)


In [14]:
#die Spalten habe ich leider mit Leerzwichen benannt, sau dumm..... 
#daher behebe ich den Fehler hier wieder
df = df.rename({'likes 30': 'likes_last'}, axis='columns')

In [15]:
#Ich neheme nur die Tweets die 30x aktualisiert wurden 
#und alle die beim ersten crawlen weniger als 5 retweets_counts haben damit versuchen wir die RT's auszusortieren.
#Wir nehmen alle Tweets außer die ersten 150 um die aktuellsten zu analysieren kann man wahrscheinlich  mit dem 
#TimeStamp lösen indem wir die Zeit zwischen created_at und retweets_1
#auf 120 sek. limitieren

df = df[df.counter == 30]
#df = df[df.retweets < 5]
#df = df[df.likes_last > 50]
#df = df[df.retweets_last == 0]
df = df[df.text.str.contains('RT', case=True, regex=True) == False]
#df = df.tail(100)
df = df[df.name != 'janboehm']

#drop the duplicates 
df = df[df.id.duplicated() == False ]
len(df)


12970

## Engagement Rate 
Engagement rate wird für jeden Supernode berechnet, in ein json doc. gespeichert um dies für die spätere prediction bzw. klassifizierung zuverwenden.

Desweitern wird untersucht ob die Engagement rate mit der Follower anzhal des Supernodes korreliert. Dies wird auch grafisch dargestellt. 

In [16]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

%matplotlib inline

#Durchschnittliche likes höher ermitteln 
df_likes = df.groupby('name')['likes_last'].mean()
df_likes = df_likes.to_frame()

#Durchschnittliche retweets höher ermitteln 
df_re = df.groupby('name')['retweets 30'].mean()
df_re = df_re.to_frame()

#Feststellen wie viel Follower ein Twitternutzer maximal hat
df_fol = df.groupby('name')['followers'].max()
df_fol = df_fol.to_frame()


#Follower / avg. likes = engagement rate 
df_engagement_likes= pd.Series((df_likes['likes_last']/df_fol['followers'])* 100, name="engagement_likes")
df_engagement_likes = df_engagement_likes.round(decimals=6)
df_engagement_likes = df_engagement_likes.to_frame()

#Follower / avg. retweets = engagement rate 
df_engagement_retweets = pd.Series((df_re['retweets 30']/df_fol['followers'])* 100, name="engagement_retweets")
df_engagement_retweets = df_engagement_retweets.round(decimals=6)
df_engagement_retweets = df_engagement_retweets.to_frame()


#Die verschiedenen DF mergen, erstellen eines DF mit followern, retweets (mean im letzten Intervall) und sich der 
#darau ergebene engagement rate 
df_res = pd.concat([df_fol, df_re,df_likes, df_engagement_likes,df_engagement_retweets], axis=1, sort=False)
display(df_res)

# write df_res to disk so that we can use it later on as a reference to evaluate if Tweet 
# is expexted to be above engagement rate 
df_res.to_json('./engagement_rate/engagement_rate')

#Regression analysis
x = df_res['followers']
y = df_res['engagement_likes']
results = sm.OLS(y,sm.add_constant(x)).fit()
print(results.summary())

x = df_res['followers']
y = df_res['engagement_retweets']
results = sm.OLS(y,sm.add_constant(x)).fit()
print(results.summary())


import plotly.express as px

#df = px.data.gapminder().query("year==2007 and continent=='Americas'")

fig = px.scatter(df_res, x="engagement_likes", y="followers", text=df_res.index, log_x=True, size_max=60)

fig.update_traces(textposition='top center')

fig.update_layout(
    height=800,
    title_text='Engagement like'
)

fig.show()

fig = px.scatter(df_res, x="engagement_retweets", y="followers", text=df_res.index, log_x=True, size_max=60)

fig.update_traces(textposition='top center')

fig.update_layout(
    height=800,
    title_text='Engagement Retweets'
)

fig.show()

,followers,retweets 30,likes_last,engagement_likes,engagement_retweets
name,,,,,
ArminWolf,433030,3.731928,34.286145,0.007918,0.000862
BMI_OE,13779,1.326087,7.717391,0.056008,0.009624
DiePressecom,58751,0.305732,0.878981,0.001496,0.000520
DieterBornemann,32198,1.538462,7.269231,0.022577,0.004778
EvaDichand,7830,0.181818,4.136364,0.052827,0.002322
HaegeleVincent,3,0.000000,0.000000,0.000000,0.000000
Heute_at,14771,0.213043,0.557391,0.003774,0.001442
HubertSickinger,44969,0.284337,3.178313,0.007068,0.000632
IngridThurnher,212493,1.660377,19.962264,0.009394,0.000781


                            OLS Regression Results                            
Dep. Variable:       engagement_likes   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     5.193
Date:                Wed, 05 Feb 2020   Prob (F-statistic):             0.0293
Time:                        14:56:52   Log-Likelihood:                 80.149
No. Observations:                  35   AIC:                            -156.3
Df Residuals:                      33   BIC:                            -153.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0323      0.005      6.010      0.0

## Likes Analyse 

Analyse wie sich die anzahl von likes über die Zeit entwickelt 

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
# Outliers rausschmeißen und den durchschnittlichen Totalen Zuwachs plotten
#Um Datensets mit 30 Intervallen zu analysieren: 
df = df[['likes', 'likes 1', 'likes 2', 'likes 3', 'likes 4', 'likes 5', 'likes 6', 'likes 7', 'likes 8', 'likes 9', 'likes 10', 'likes 11', 'likes 12', 'likes 13', 'likes 14', 'likes 15', 'likes 16', 'likes 17', 'likes 18', 'likes 19', 'likes 20', 'likes 21', 'likes 22', 'likes 23', 'likes 24', 'likes 25', 'likes 26', 'likes 27', 'likes 28', 'likes 29', 'likes_last']].copy()
#Um Datensets mit 60 Intervallen zu analysieren: 
#df = df[['retweets','retweets 1', 'retweets 2', 'retweets 3','retweets 4','retweets 5','retweets 6','retweets 7','retweets 8','retweets 9','retweets 10','retweets 11','retweets 12','retweets 13','retweets 14','retweets 15','retweets 16','retweets 17','retweets 18','retweets_19','retweets_20','retweets 21','retweets 22','retweets 24','retweets 25','retweets 26','retweets 27','retweets 29','retweets_last','retweets 31','retweets 32','retweets 33','retweets 34','retweets 35','retweets 36','retweets 37','retweets 38','retweets 39','retweets 40','retweets 41','retweets 42','retweets 43','retweets 44','retweets 45','retweets 46','retweets 47','retweets 48','retweets 49','retweets 50','retweets 51','retweets 52','retweets 53','retweets 54','retweets 55','retweets 56','retweets 57','retweets 58','retweets 59','retweets 60']].copy()

__Outliers entfernen und Durchschnittswerte plotten__

In [18]:
t = df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() <1.5).all(axis=1)].tail(12000).mean()
fig = px.line(y=t, x = t.index,  labels={'x':'Intervall', 'y':'Likes count'})
fig.show()

__ohne die Outlieres zu entfernen__

In [19]:
#Geht auch ohne die Outliers zu entfernen da wir nur mit Durchschnittswerten rechenen
%matplotlib inline
df = df[['likes', 'likes 1', 'likes 2', 'likes 3', 'likes 4', 'likes 5', 'likes 6', 'likes 7', 'likes 8', 'likes 9', 'likes 10', 'likes 11', 'likes 12', 'likes 13', 'likes 14', 'likes 15', 'likes 16', 'likes 17', 'likes 18', 'likes 19', 'likes 20', 'likes 21', 'likes 22', 'likes 23', 'likes 24', 'likes 25', 'likes 26', 'likes 27', 'likes 28', 'likes 29', 'likes_last']].copy()
t = df.tail(12000).mean()
fig = px.line(y=t, x = t.index, labels={'x':'Intervall', 'y':'Likes count'})
fig.show()


__Prozentuale Veränderung der Zuwachsrate bei Likes__ 

In [20]:
t = df.tail(12000).mean().pct_change()
fig = px.line(y=t, x = t.index, labels={'x':'Intervall', 'y':'%-Change (Likes)'})
fig.show()

__Standart abweichung der Likes__

In [18]:
import numpy as np 
a = df.to_numpy(dtype=None, copy=False)
np.std(a)

29.988503121272515

__Warum haben wir die Grenze bei 30 Minuten gemacht?__

In [19]:
df = pd.read_json('../data/master_list_16_01_1230_60_intervals')
df = df[df.counter == 180]
df = df[df.text.str.contains('RT', case=True, regex=True) == False]
df = df[df.name != 'janboehm']

#drop the duplicates 
df = df[df.id.duplicated() == False ]

x = np.arange(181)
L_rt = []
for i in x: 
    if i == 0:
        r = 'retweets'
        L_rt.append(r)
    else:
        r = 'retweets ' + str(i)
        L_rt.append(r)


        
df = df[L_rt]
t = df.tail(300).mean()
fig = px.line(y=t, x = t.index, labels={'x':'Intervall', 'y':'Retweet count'})
fig.show()


## Retweet Analyses 

In [78]:
from scipy import stats
import numpy as np

df = pd.read_json('./merge_new.json')

df = df[df.counter == 30]
#df = df[df.retweets < 5]
#df = df[df.likes_last > 50]
#df = df[df.retweets_last == 0]
df = df[df.text.str.contains('RT', case=True, regex=True) == False]
#df = df.tail(100)
df = df[df.name != 'janboehm']

#drop the duplicates 
df = df[df.id.duplicated() == False ]



# Outliers rausschmeißen und den durchschnittlichen Totalen Zuwachs plotten
#Um Datensets mit 30 Intervallen zu analysieren: 
df = df[['retweets','retweets 1', 'retweets 2', 'retweets 3','retweets 4','retweets 5','retweets 6','retweets 7','retweets 8','retweets 9','retweets 10','retweets 11','retweets 12','retweets 13','retweets 14','retweets 15','retweets 16','retweets 17','retweets 18','retweets 19','retweets 20','retweets 21','retweets 22','retweets 24','retweets 25','retweets 26','retweets 27','retweets 29','retweets 30']].copy()
#Um Datensets mit 60 Intervallen zu analysieren: 
#df = df[['retweets','retweets 1', 'retweets 2', 'retweets 3','retweets 4','retweets 5','retweets 6','retweets 7','retweets 8','retweets 9','retweets 10','retweets 11','retweets 12','retweets 13','retweets 14','retweets 15','retweets 16','retweets 17','retweets 18','retweets_19','retweets_20','retweets 21','retweets 22','retweets 24','retweets 25','retweets 26','retweets 27','retweets 29','retweets_last','retweets 31','retweets 32','retweets 33','retweets 34','retweets 35','retweets 36','retweets 37','retweets 38','retweets 39','retweets 40','retweets 41','retweets 42','retweets 43','retweets 44','retweets 45','retweets 46','retweets 47','retweets 48','retweets 49','retweets 50','retweets 51','retweets 52','retweets 53','retweets 54','retweets 55','retweets 56','retweets 57','retweets 58','retweets 59','retweets 60']].copy()



__Retweet Outliers entfernen und Durchschnittswerte plotten__

In [79]:
t = df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() <1.5).all(axis=1)].tail(12000).mean()
fig = px.line(y=t, x = t.index,  labels={'x':'Intervall', 'y':'Retweets count'})
fig.show()

__Outlieres nicht entfernt__

In [80]:
t = df.tail(12000).mean()
fig = px.line(y=t, x = t.index, labels={'x':'Intervall', 'y':'Retweets count'})
fig.show()

In [81]:
t = df.tail(12000).mean().pct_change()
fig = px.line(y=t, x = t.index, labels={'x':'Intervall', 'y':'%-Change (Retweets)'})
fig.show()

__Standartabweichung Retweets__

In [56]:
import numpy as np 
a = df.to_numpy(dtype=None, copy=False)
np.std(a)

3.706351792120594